In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

In [2]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [3]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool

In [4]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
    api_version=os.environ["API_VERSION"],
    model=os.environ["MODEL_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    # api_key="sk-...", # For key-based authentication.
)

In [6]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [7]:
bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"]
)

conn_id = bing_connection.id

HttpResponseError: (AuthorizationFailed) The client 'vgiraud@microsoft.com' with object id '65abf6bf-35c2-4a53-8433-f2afcc4b5a20' does not have authorization to perform action 'Microsoft.MachineLearningServices/workspaces/connections/read' over scope '/subscriptions/a2adbedd-e5e9-44a5-ba32-6b2175e3018a/resourceGroups/rg-sc-vgi/providers/Microsoft.MachineLearningServices/workspaces/prj-sc-vgi/connections/binggroundingscvgi' or the scope is invalid. If access was recently granted, please refresh your credentials.
Code: AuthorizationFailed
Message: The client 'vgiraud@microsoft.com' with object id '65abf6bf-35c2-4a53-8433-f2afcc4b5a20' does not have authorization to perform action 'Microsoft.MachineLearningServices/workspaces/connections/read' over scope '/subscriptions/a2adbedd-e5e9-44a5-ba32-6b2175e3018a/resourceGroups/rg-sc-vgi/providers/Microsoft.MachineLearningServices/workspaces/prj-sc-vgi/connections/binggroundingscvgi' or the scope is invalid. If access was recently granted, pleas

In [ ]:
async def web_ai_agent(query: str) -> str:
    print("This is Bing for Azure AI Agent Service .......")
    bing = BingGroundingTool(connection_id=conn_id)
    with project_client:
        agent = project_client.agents.create_agent(
            model=os.environ["MODEL_DEPLOYMENT_NAME"],
            name="my-assistant",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
        print(f"Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query,
        )
        print(f"SMS: {message}")
        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
        project_client.agents.delete_agent(agent.id)
        print("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])
    return messages["data"][0]["content"][0]["text"]["value"]



In [ ]:
bing_search_agent = AssistantAgent(
    name="assistant",
    model_client=az_model_client,
    tools=[web_ai_agent],
    system_message="Use tools to solve tasks.",
)

In [ ]:
bing = BingGroundingTool(connection_id=conn_id)

In [ ]:
async def assistant_run() -> None:
    response = await bing_search_agent.on_messages(
            [TextMessage(content="What is the latest Microsoft stock price?", source="user")],
            cancellation_token=CancellationToken(),
    )
    # print(response.inner_messages)
    print(response.chat_message)

In [ ]:
await assistant_run()